In [23]:
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_watson.natural_language_understanding_v1 import *
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

nlu_credentials = {}
with open('./NLUapikey.txt', 'r') as f:
    for ln in f.readlines():
        k, v = ln.split()
        nlu_credentials[k] = v

authenticator = IAMAuthenticator(nlu_credentials['apikey'])
nlu = NaturalLanguageUnderstandingV1(version='2019-07-12', authenticator=authenticator)
nlu.set_service_url(nlu_credentials['url'])

def call_nlu(text, features, key):
    response = nlu.analyze(text=text, features=features).get_result()
    return response[key]


# エンティティ抽出機能

In [29]:
text = "安倍首相はトランプ氏と昨日，大阪の国際会議場で会談した．"
features = Features(entities=EntitiesOptions())
ret = call_nlu(text, features, 'entities')
print(json.dumps(ret, indent=2, ensure_ascii=False))

[
  {
    "type": "Person",
    "text": "トランプ氏",
    "relevance": 0.953262,
    "count": 1,
    "confidence": 0.917404
  },
  {
    "type": "Date",
    "text": "昨日",
    "relevance": 0.784743,
    "count": 1,
    "confidence": 0.952695
  },
  {
    "type": "Person",
    "text": "安倍",
    "relevance": 0.703143,
    "count": 1,
    "confidence": 0.993651
  },
  {
    "type": "JobTitle",
    "text": "首相",
    "relevance": 0.570908,
    "count": 1,
    "confidence": 0.730175
  },
  {
    "type": "Facility",
    "text": "国際会議場",
    "relevance": 0.447856,
    "count": 1,
    "confidence": 0.536862
  },
  {
    "type": "Location",
    "text": "大阪",
    "relevance": 0.287359,
    "count": 1,
    "confidence": 0.97182
  }
]


# type一覧
Date：日付<br>
Duration：期間<br>
EmailAddress:eメールアドレス<br>
Fcility:施設名<br>
GoegraphicFeature:地理名<br>
Hashtag:ハッシュタグ<br>
IPAddress:IPアドレス<br>
jobTitle:役職名<br>
Location:地名<br>
Measure:単位付きの量<br>
Money:金額<br>
Ordinal:順序<br>
Organization:組織名<br>
Person:人名<br>
Time:時間<br>
TwitterHandle:ツイッターハンドル名<br>

# 関係抽出機能

In [30]:
text = 'このイベントは東京の国立競技場で開催されました．'
features = Features(relations=RelationsOptions())
ret = call_nlu(text, features, 'relations')
print(json.dumps(ret, indent=2, ensure_ascii=False))

[
  {
    "type": "locatedAt",
    "sentence": "このイベントは東京の国立競技場で開催されました.",
    "score": 0.910188,
    "arguments": [
      {
        "text": "国立競技場",
        "location": [
          10,
          15
        ],
        "entities": [
          {
            "type": "Facility",
            "text": "国立競技場"
          }
        ]
      },
      {
        "text": "東京",
        "location": [
          7,
          9
        ],
        "entities": [
          {
            "type": "Location",
            "text": "東京"
          }
        ]
      }
    ]
  }
]


# type一覧
basedIn:組織Aは場所Bを本拠地とする<br>
locatedAt:施設Aは場所Bに存在する<br>
employedBy:人Aは組織Bに雇用されている<br>
managerOf:人Aは人Bの所属長である<br>
partOf:エンティティAはエンティティBの一部である

# 評判分析機能

In [34]:
text1 = 'やはりバンプはすごいです．新曲は歌詞もメロディーも心に響きます．'
text2 = '何が言いたいのかよくわからないです．さらにコード進行がめちゃくちゃ．'

features = Features(sentiment=SentimentOptions())

ret = call_nlu(text1, features, 'sentiment')
print(json.dumps(ret, indent=2, ensure_ascii=False))

ret = call_nlu(text2, features, 'sentiment')
print(json.dumps(ret, indent=2, ensure_ascii=False))

{
  "document": {
    "score": 0.971189,
    "label": "positive"
  }
}
{
  "document": {
    "score": -0.665101,
    "label": "negative"
  }
}


# キーワード抽出機能

In [38]:
text = 'ながぬま温泉は北海道でも屈指の湯量を誇り，加水・加温はせずに100%源泉かけ流しで，保温効果が高く湯冷めしにくい塩化物泉であり，「熱の湯」とも呼ばれ，保養や療養を目的として多くの方が訪れている．'

features = Features(keywords=KeywordsOptions(limit=5))

ret = call_nlu(text, features, 'keywords')

print(json.dumps(ret, indent=2, ensure_ascii=False))

[
  {
    "text": "ながぬま温泉",
    "relevance": 0.922948,
    "count": 1
  },
  {
    "text": "加水・加温",
    "relevance": 0.764238,
    "count": 1
  },
  {
    "text": "屈指の湯量",
    "relevance": 0.707494,
    "count": 1
  },
  {
    "text": "保温効果",
    "relevance": 0.621574,
    "count": 1
  },
  {
    "text": "塩化物泉",
    "relevance": 0.567746,
    "count": 1
  }
]


# 概念分析機能

In [41]:
text = 'ながぬま温泉は北海道でも屈指の湯量を誇り，加水・加温はせずに100%源泉かけ流しで，保温効果が高く湯冷めしにくい塩化物泉であり，「熱の湯」とも呼ばれ，保養や療養を目的として多くの方が訪れている．'

features = Features(concepts=ConceptsOptions(limit=3))
ret = call_nlu(text, features, 'concepts')
print(json.dumps(ret, indent=2, ensure_ascii=False))

[
  {
    "text": "湧出量",
    "relevance": 0.824821,
    "dbpedia_resource": "http://ja.dbpedia.org/resource/湧出量"
  },
  {
    "text": "湯の花",
    "relevance": 0.670132,
    "dbpedia_resource": "http://ja.dbpedia.org/resource/湯の花"
  },
  {
    "text": "地獄温泉",
    "relevance": 0.670132,
    "dbpedia_resource": "http://ja.dbpedia.org/resource/地獄温泉"
  }
]


# カテゴリ分類機能

In [44]:
text = '自然環境の保護を図るとともに，地域に調和した温泉利用施設を維持し，豊かさとふれあいのある保養の場とする．'

features = Features(categories=CategoriesOptions())
ret = call_nlu(text, features, 'categories')
print(json.dumps(ret, indent=2, ensure_ascii=False))

[
  {
    "score": 0.652046,
    "label": "/sports/skiing"
  },
  {
    "score": 0.641494,
    "label": "/travel/hotels"
  },
  {
    "score": 0.580031,
    "label": "/science/ecology/environmental disaster"
  }
]


# 意味役割抽出機能

In [46]:
text = 'bumpの歌詞は聞く人によって様々な解釈が可能である．'

features = Features(semantic_roles=SemanticRolesOptions())
ret = call_nlu(text, features, 'semantic_roles')
print(json.dumps(ret, indent=2, ensure_ascii=False))

[
  {
    "subject": {
      "text": "bumpの歌詞は"
    },
    "sentence": "bumpの歌詞は聞く人によって様々な解釈が可能である.",
    "action": {
      "verb": {
        "text": "聞く"
      },
      "text": "聞く",
      "normalized": "聞く"
    }
  }
]
